In [ ]:
import matplotlib.pyplot as plt
import datetime
import tilemapbase as tmb
import numpy as np
import pandas as pd
from pluma.stream.georeference import Georeference
from pluma.stream.ubx import _UBX_MSGIDS
from pluma.schema import Dataset

## Notebook plotting
%matplotlib inline
plt.style.use('ggplot')

## Figure export parameters
new_rc_params = {'text.usetex': False,
"svg.fonttype": 'none'
}
import matplotlib as mpl
mpl.rcParams.update(new_rc_params)

In [ ]:
## Important notes
# It appears the the end of the video file (.avi) is corrupted due to the decoding process. As a result we should crop the 0mq data to the same length as the video file.

In [ ]:
stream_root_folder = r'C:\Users\neurogears\Desktop\2023_05_15_23_35_14'
from benchmark_schemas import build_benchmarkschema_pupillabs

dataset = Dataset(
    stream_root_folder,
    datasetlabel="PupilLabs_Benchmark",
    georeference= Georeference(),
    schema=build_benchmarkschema_pupillabs)
dataset.populate_streams(autoload=True)



In [ ]:
## Check corrupted frames
video_path = stream_root_folder + r'\pupil_video.avi'
import cv2

cap = cv2.VideoCapture(video_path)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # returns the correct number of frames
print('Number of frames: ', frame_count)

In [ ]:
decoded_counter = dataset.streams.PupilLabs.Counter.DecodedFrames
#decoded_counter = dataset.streams.PupilLabs.Counter.RawFrames

decoded_counter.data = decoded_counter.data[~decoded_counter.data.index.duplicated(keep='first')]
decoded_counter.data = decoded_counter.data[~(decoded_counter.data == 1)]

## Load luminance data tracked from the video
lum = pd.read_csv(stream_root_folder + r'\pupil_video_lum.csv', header=None, names=['lum'])
decoded_counter.data["lum"] = lum.values
decoded_counter.data

In [ ]:
##Find rising edges in the video
threshold = 70
ttl = (decoded_counter.data['lum'] > threshold).astype(int)
rising_edge_camera = decoded_counter.data.index[ttl.diff() == 1]


## find rising edges in harp
dataset.streams.BioData.Set.data = dataset.streams.BioData.Set.data[~dataset.streams.BioData.Set.data.index.duplicated(keep='first')]
bit_mask = 1024
rising_edge_harp = dataset.streams.BioData.Set.data[dataset.streams.BioData.Set.data & bit_mask > 0].index.values
print((len(rising_edge_camera), len(rising_edge_harp)))

## There seems to be an extra ttl at the start and many at the end. Lets realign the data...
rising_edge_harp = np.delete(rising_edge_harp, 0)
rising_edge_harp = rising_edge_harp[0:len(rising_edge_camera)]
print((len(rising_edge_camera), len(rising_edge_harp)))


In [ ]:
plt.figure(figsize= (9,6))

plt.plot(np.diff(rising_edge_camera) / 1e9, label='camera')
plt.plot(np.diff(rising_edge_harp) / 1e9, label='harp')
plt.show()

delta_t = rising_edge_camera - rising_edge_harp
delta_t = -delta_t / np.timedelta64(1, 'ms')

plt.figure(figsize= (9,6))
plt.plot(delta_t)
plt.show()





plt.figure(figsize= (9,6))
plt.hist(delta_t, bins = 25, color= "Teal")
plt.xlabel('Latency (ms)')
plt.ylabel('Counts')
#plt.xlim((0,5000))
plt.title(f'Pupil tracking latency benchmark\n$\mu = {np.mean(delta_t):.2f}, \sigma = {np.std(delta_t):.2f},$ [min:max] = [{np.min(delta_t):.2f}:{np.max(delta_t):.2f}]')
plt.show()
